In [ ]:
#!pip install -q transformers accelerate bitsandbytes sentence-transformers datasets scipy torch torchaudio tqdm


In [ ]:
"""
PROJECT NIKA: PHASE 6 - INFORMATION GEOMETRY GROUNDWORK
================================================================
"The Curvature of Meaning"

OBJECTIVE:
Measure the "Statistical Distance" (KL Divergence) between concepts to
construct the Riemannian Metric of the model's thought process.

METHOD:
1. Extract Probability Distributions for Concept A and Concept B.
2. Calculate KL Divergence (Information Distance) vs Cosine Distance (Vector Distance).
3. If KL Divergence >> Cosine Distance, the space is CURVED (Manifold verified).
"""

import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForCausalLM

# ============================================================================
# SECTION 1: THE MANIFOLD METRICS
# ============================================================================

def get_distributions(model, tokenizer, text_input):
    """
    Returns the Probability Distribution (Logits) over the entire vocabulary.
    This represents the 'Point' on the Statistical Manifold.
    """
    inputs = tokenizer(text_input, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the logits of the last token (the "Next Token Prediction" state)
    next_token_logits = outputs.logits[0, -1, :]

    # Convert to Probability Distribution (Softmax)
    probs = F.softmax(next_token_logits, dim=-1)
    return probs, inputs['input_ids']

def calculate_metrics(probs_p, probs_q):
    """
    Calculates two distances:
    1. Euclidean/Cosine (Flat Space)
    2. KL Divergence (Curved Information Space)
    """
    # 1. Euclidean Distance (L2 norm between probability vectors)
    euclidean = torch.dist(probs_p, probs_q, p=2).item()

    # 2. KL Divergence (Information Geometry Metric)
    # D_KL(P || Q) = sum(P(x) * log(P(x) / Q(x)))
    # We add a small epsilon to avoid log(0)
    epsilon = 1e-9
    kl_div = F.kl_div((probs_q + epsilon).log(), probs_p, reduction='batchmean').item()

    return euclidean, kl_div

# ============================================================================
# SECTION 2: THE CURVATURE EXPERIMENT
# ============================================================================

def probe_manifold_curvature(model, tokenizer):
    print("="*60)
    print("PHASE 6: PROBING THE MANIFOLD CURVATURE")
    print("="*60)

    # We test 3 Concept Pairs
    pairs = [
        ("King", "Queen"),       # Semantic Twins (Should be close in both)
        ("Logic", "Emotion"),    # Semantic Opposites (Far in both)
        ("Rose", "Jaguar")       # Context Dependent (Rose=Flower? Jaguar=Car/Animal?)
    ]

    results = []

    for concept_a, concept_b in pairs:
        print(f"\nAnalyzing Pair: {concept_a} <-> {concept_b}")

        # 1. Get Manifold Coordinates (Distributions)
        p_dist, _ = get_distributions(model, tokenizer, f"The concept of {concept_a} implies")
        q_dist, _ = get_distributions(model, tokenizer, f"The concept of {concept_b} implies")

        # 2. Measure Distances
        euc_dist, kl_dist = calculate_metrics(p_dist, q_dist)

        # 3. Calculate Curvature Ratio
        # If Space is Flat, KL and Euclidean scale linearly.
        # High Ratio means "Hidden Information Distance" (Curvature).
        curvature_ratio = kl_dist / (euc_dist + 1e-9)

        print(f"  > Euclidean Distance: {euc_dist:.4f}")
        print(f"  > Information Dist (KL): {kl_dist:.4f}")
        print(f"  > Curvature Ratio:    {curvature_ratio:.4f}")

        results.append((concept_a, concept_b, curvature_ratio))

    return results

# ============================================================================
# SECTION 3: EXECUTION
# ============================================================================

def run_phase6_groundwork():
    # Load Model (Using the same NIKA checkpoints)
    model_name = "Qwen/Qwen2.5-7B-Instruct"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto"
    )

    results = probe_manifold_curvature(model, tokenizer)

    print("\n" + "="*60)
    print("MANIFOLD ANALYSIS COMPLETE")
    print("="*60)
    print("Interpretation:")
    print("- Low Ratio (~1-5): Flat Space. Linear Algebra works (Phase 5).")
    print("- High Ratio (>10): Curved Space. Geodesics required (Phase 6).")

    for r in results:
        print(f"{r[0]} <-> {r[1]}: Ratio {r[2]:.2f}")

if __name__ == "__main__":
    run_phase6_groundwork()

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

KeyboardInterrupt: 

In [ ]:
"""
PROJECT NIKA: PHASE 6 - ROBUST MANIFOLD PROBE (FIXED)
================================================================
Fixes 'nan' errors by enforcing Float32 precision for manifold metrics.
"""

import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM

# ============================================================================
# SECTION 1: ROBUST METRICS ENGINE
# ============================================================================

def get_distributions_robust(model, tokenizer, text_input):
    """
    Extracts probabilities but casts to FLOAT32 to prevent underflow.
    """
    inputs = tokenizer(text_input, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)

    # Get logits of the last token
    next_token_logits = outputs.logits[0, -1, :]

    # CRITICAL FIX: Cast to float32 before softmax to preserve tiny values
    next_token_logits = next_token_logits.float()

    # Softmax to get probabilities (sum to 1.0)
    probs = F.softmax(next_token_logits, dim=-1)
    return probs

def calculate_metrics_robust(probs_p, probs_q):
    """
    Calculates metrics with safety clamps for log(0).
    """
    # 1. Euclidean Distance (L2)
    # Simple straight-line distance in probability space
    euclidean = torch.norm(probs_p - probs_q, p=2).item()

    # 2. KL Divergence (Information Distance)
    # Formula: sum(P * log(P / Q))
    # We implement manually for maximum stability

    epsilon = 1e-8 # Safety buffer (10x smaller than typical float16 limit)

    # Clamp probabilities to avoid log(0)
    p_safe = probs_p.clamp(min=epsilon)
    q_safe = probs_q.clamp(min=epsilon)

    # Calculate KL: sum( p * (log(p) - log(q)) )
    kl_div = torch.sum(p_safe * (torch.log(p_safe) - torch.log(q_safe))).item()

    return euclidean, kl_div

# ============================================================================
# SECTION 2: THE CURVATURE EXPERIMENT
# ============================================================================

def probe_manifold_curvature_robust():
    print("="*60)
    print("PHASE 6: ROBUST MANIFOLD PROBE")
    print("="*60)

    # Load Model (Same as before)
    model_name = "Qwen/Qwen2.5-7B-Instruct"
    try:
        # Check if model is already loaded in global scope to save time
        if 'model' not in globals():
            print("Loading Model...")
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model = AutoModelForCausalLM.from_pretrained(
                model_name,
                torch_dtype=torch.float16,
                device_map="auto"
            )
        else:
            print("Using existing model from memory.")

    except Exception as e:
        # Fallback for fresh run
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16,
            device_map="auto"
        )

    # Test Pairs
    pairs = [
        ("King", "Queen"),       # Close Semantic Sibling
        ("Logic", "Emotion"),    # Distant Semantic Opposite
        ("Rose", "Jaguar"),      # Ambiguous/Distant
        ("One", "Two")           # Numeric Sequence (Very Close)
    ]

    print("-" * 60)
    print(f"{'PAIR':<20} | {'EUCLIDEAN':<10} | {'KL DIV (INFO)':<15} | {'RATIO':<10}")
    print("-" * 60)

    results = []

    for concept_a, concept_b in pairs:
        # Prompt: "The concept of X implies" creates a distribution vector for X
        p_dist = get_distributions_robust(model, tokenizer, f"The concept of {concept_a} implies")
        q_dist = get_distributions_robust(model, tokenizer, f"The concept of {concept_b} implies")

        euc, kl = calculate_metrics_robust(p_dist, q_dist)

        # Ratio of Info Distance to Linear Distance
        # High Ratio = High Curvature (Information density is higher than linear space suggests)
        # Ratio > 10 suggests we need Geodesics (Phase 6).
        ratio = kl / (euc + 1e-9)

        print(f"{concept_a:<9} <-> {concept_b:<8} | {euc:<10.4f} | {kl:<15.4f} | {ratio:<10.2f}")
        results.append((concept_a, concept_b, ratio))

    print("-" * 60)
    return results

if __name__ == "__main__":
    probe_manifold_curvature_robust()

In [ ]:
"""
PROJECT NIKA: PHASE 6 - THE GEODESIC NAVIGATOR
================================================================
"Traversing the Manifold"

OBJECTIVE:
Connect distinct semantic concepts by following the path of
maximum probability and minimum geometric distance (The Geodesic),
rather than linear interpolation.
"""

import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM

class GeodesicNavigator:
    def __init__(self, model_name="Qwen/Qwen2.5-7B-Instruct"):
        print(f"Initializing Geodesic Navigator with {model_name}...")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        self.device = self.model.device

    def get_embedding(self, word):
        """Get the static embedding vector for a concept"""
        inputs = self.tokenizer(word, return_tensors="pt").to(self.device)
        with torch.no_grad():
            outputs = self.model(**inputs, output_hidden_states=True)
        # Use the last hidden state of the last token
        return outputs.hidden_states[-1][0, -1, :]

    def find_geodesic(self, start_concept, target_concept, max_steps=10):
        print(f"\n--- TRACING GEODESIC: {start_concept} -> {target_concept} ---")

        path = [start_concept]
        current_word = start_concept

        # 1. Get Target Gravity (The Embedding of the Goal)
        target_vec = self.get_embedding(target_concept)
        target_vec = F.normalize(target_vec, dim=0)

        for step in range(max_steps):
            # 2. Get Manifold Options (Next Token Probabilities)
            inputs = self.tokenizer(f"The concept of {current_word} leads to", return_tensors="pt").to(self.device)
            with torch.no_grad():
                outputs = self.model(**inputs)

            next_token_logits = outputs.logits[0, -1, :]

            # Filter for top 50 semantically viable moves (The "Probabilistic Ridge")
            top_k_probs, top_k_ids = torch.topk(F.softmax(next_token_logits, dim=-1), 50)

            best_next_word = None
            best_score = -float('inf')

            # 3. Calculate Geodesic Step (Prob * Direction)
            for i, token_id in enumerate(top_k_ids):
                word = self.tokenizer.decode(token_id).strip()

                # Filter trivial words
                if not word.isalpha() or len(word) < 3 or word.lower() in [w.lower() for w in path]:
                    continue

                # A. Manifold Support (How natural is this step?)
                prob_score = top_k_probs[i].item()

                # B. Vector Direction (Are we getting closer to the target?)
                # We interpret the word to get its vector
                candidate_vec = self.get_embedding(word)
                candidate_vec = F.normalize(candidate_vec, dim=0)

                # Cosine Similarity to Target
                direction_score = torch.dot(candidate_vec, target_vec).item()

                # THE GEODESIC FORMULA:
                # Maximize (Probability * Weight) + (Direction * Weight)
                # We weight Direction heavily to force traversal
                total_score = (prob_score * 0.3) + (direction_score * 0.7)

                if total_score > best_score:
                    best_score = total_score
                    best_next_word = word

            if not best_next_word:
                print("Path blocked (Local Minima).")
                break

            path.append(best_next_word)
            current_word = best_next_word
            print(f"Step {step+1}: {best_next_word} (Score: {best_score:.4f})")

            # Check proximity (Simple string check for convergence)
            if best_next_word.lower() == target_concept.lower() or step == max_steps - 1:
                break

        return path

# ============================================================================
# EXECUTION
# ============================================================================

def run_phase6_geodesic():
    navigator = GeodesicNavigator()

    # Test 1: The High Curvature Path (Logic -> Emotion)
    # Linear interpolation fails here. Does Geodesic work?
    navigator.find_geodesic("Logic", "Emotion")

    # Test 2: The Abstract Path (Chaos -> Order)
    navigator.find_geodesic("Chaos", "Order")

    # Test 3: The Creative Path (Night -> Day)
    navigator.find_geodesic("Night", "Day")

if __name__ == "__main__":
    run_phase6_geodesic()

Initializing Geodesic Navigator with Qwen/Qwen2.5-7B-Instruct...


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]


--- TRACING GEODESIC: Logic -> Emotion ---
Step 1: the (Score: 0.4754)
Step 2: important (Score: 0.4331)
Step 3: understand (Score: 0.4272)
Step 4: learning (Score: 0.4409)
Step 5: various (Score: 0.4292)
Step 6: different (Score: 0.4301)
Step 7: differentiation (Score: 0.4317)
Step 8: powerful (Score: 0.4340)
Step 9: stronger (Score: 0.4245)
Step 10: weaker (Score: 0.4758)

--- TRACING GEODESIC: Chaos -> Order ---
Step 1: the (Score: 0.5534)
Step 2: such (Score: 0.4957)
Step 3: higher (Score: 0.4866)
Step 4: which (Score: 0.4810)
Step 5: that (Score: 0.4613)
Step 6: and (Score: 0.4624)
Step 7: important (Score: 0.4762)
Step 8: prove (Score: 0.4831)
Step 9: another (Score: 0.4641)
Step 10: reflection (Score: 0.4808)

--- TRACING GEODESIC: Night -> Day ---
Step 1: Day (Score: 0.7005)


In [ ]:
"""
PROJECT NIKA: PHASE 6 - THE EPISTEMIC GEODESIC NAVIGATOR (ROBUST)
================================================================
"The Path of Clarity"

OBJECTIVE:
Traverse the manifold by minimizing 'Epistemic Effort' (Entropy).
This implements the "Minimal Cognitive Path" approach:
Geodesic = minimal transformation resolving internal contradiction.

FORMULA:
Score = (Manifold_Support * Target_Alignment) / Epistemic_Resistance
"""

import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM

# ============================================================================
# SECTION 1: ROBUST INITIALIZATION (Fixing the NameError)
# ============================================================================

def get_or_load_model():
    """
    Checks if model/tokenizer exist in global scope.
    If not, loads them automatically.
    """
    global model, tokenizer

    print("Checking model status...")

    if 'model' in globals() and 'tokenizer' in globals():
        print("✓ Found existing model in memory. Reusing.")
        return model, tokenizer

    print("⚠ Model not found. Loading Qwen2.5-7B-Instruct (Float16)...")
    try:
        model_name = "Qwen/Qwen2.5-7B-Instruct"
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        print("✓ Model loaded successfully.")
        return model, tokenizer
    except Exception as e:
        print(f"CRITICAL ERROR LOADING MODEL: {e}")
        return None, None

# ============================================================================
# SECTION 2: THE EPISTEMIC NAVIGATOR
# ============================================================================

class EpistemicNavigator:
    def __init__(self):
        # Auto-load resources
        self.model, self.tokenizer = get_or_load_model()
        self.device = self.model.device
        print(f"✓ Epistemic Navigator active on {self.device}")

    def get_embedding(self, word):
        """Extracts the high-dimensional concept vector"""
        inputs = self.tokenizer(word, return_tensors="pt").to(self.device)
        with torch.no_grad():
            outputs = self.model(**inputs, output_hidden_states=True)
        return outputs.hidden_states[-1][0, -1, :].float()

    def calculate_entropy(self, probs):
        """
        Calculates the Shannon Entropy of the next-token distribution.
        High Entropy = High Confusion/Resistance (Bad Step).
        Low Entropy = High Clarity (Good Step).
        """
        # H(x) = -sum(p * log(p))
        # Add epsilon to avoid log(0)
        return -torch.sum(probs * torch.log(probs + 1e-9)).item()

    def find_geodesic(self, start_concept, target_concept, max_steps=10):
        print(f"\n--- TRACING EPISTEMIC PATH: {start_concept} ➔ {target_concept} ---")
        print(f"{'STEP':<5} | {'WORD':<15} | {'ENTROPY (Resistance)':<20} | {'SCORE (Efficiency)':<10}")
        print("-" * 75)

        path = [start_concept]
        current_word = start_concept
        target_vec = F.normalize(self.get_embedding(target_concept), dim=0)

        for step in range(max_steps):
            # 1. Probe the Manifold (Where are we?)
            prompt = f"The concept of {current_word} implies"
            inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)

            with torch.no_grad():
                outputs = self.model(**inputs)

            logits = outputs.logits[0, -1, :].float()
            probs = F.softmax(logits, dim=-1)

            # 2. Filter Candidates (Top 50 Probable Moves)
            top_k_probs, top_k_ids = torch.topk(probs, 50)

            best_next_word = None
            best_score = -float('inf')
            selected_entropy = 0.0

            for i, token_id in enumerate(top_k_ids):
                word = self.tokenizer.decode(token_id).strip().lower()

                # Filter noise, short words, and loops
                if not word.isalpha() or len(word) < 3 or word in [w.lower() for w in path]:
                    continue

                # --- THE GITA METRIC CALCULATION ---

                # A. Direction (Alignment with Goal)
                candidate_vec = F.normalize(self.get_embedding(word), dim=0)
                alignment = torch.dot(candidate_vec, target_vec).item()

                # B. Epistemic Effort (Entropy Lookahead)
                # We simulate taking this step: Does it lead to confusion?
                with torch.no_grad():
                     future_inputs = self.tokenizer(f"{prompt} {word}", return_tensors="pt").to(self.device)
                     future_out = self.model(**future_inputs)
                     future_probs = F.softmax(future_out.logits[0, -1, :].float(), dim=-1)
                     epistemic_resistance = self.calculate_entropy(future_probs)

                # C. Manifold Support
                probability = top_k_probs[i].item()

                # SCORE FORMULA: (Alignment * Probability) / Resistance
                # We look for High Alignment, High Probability, Low Resistance
                efficiency_score = (alignment * probability) / (epistemic_resistance + 0.1)

                if efficiency_score > best_score:
                    best_score = efficiency_score
                    best_next_word = word
                    selected_entropy = epistemic_resistance

            if not best_next_word:
                print("Path blocked (Local Minima).")
                break

            path.append(best_next_word)
            current_word = best_next_word

            # Visualizing the "Effort" vs "Efficiency"
            print(f" {step+1:<5} | {best_next_word:<15} | {selected_entropy:<20.4f} | {best_score:<10.4f}")

            # Convergence Check
            if current_word.lower() == target_concept.lower(): break

        return path

# ============================================================================
# EXECUTION LOOP
# ============================================================================

if __name__ == "__main__":
    # This will load the model if it's missing, or reuse it if it's there
    navigator = EpistemicNavigator()

    # 1. The High Resistance Path (Logic -> Emotion)
    navigator.find_geodesic("Logic", "Emotion")

    # 2. The Abstract Path (Chaos -> Order)
    navigator.find_geodesic("Chaos", "Order")

    # 3. The Creative Path (Night -> Day)
    navigator.find_geodesic("Night", "Day")

Checking model status...
⚠ Model not found. Loading Qwen2.5-7B-Instruct (Float16)...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✓ Model loaded successfully.
✓ Epistemic Navigator active on cuda:0

--- TRACING EPISTEMIC PATH: Logic ➔ Emotion ---
STEP  | WORD            | ENTROPY (Resistance) | SCORE (Efficiency)
---------------------------------------------------------------------------
 1     | the             | 4.9767               | 0.0263    
 2     | connect         | 0.0086               | 0.0613    
 3     | that            | 4.7939               | 0.0356    
 4     | there           | 1.4988               | 0.0043    
 5     | existence       | 2.7687               | 0.0121    
 6     | more            | 0.1948               | 0.0152    
 7     | having          | 0.2939               | 0.0064    
 8     | which           | 1.4254               | 0.0012    
 9     | equality        | 2.1713               | 0.0002    
 10    | sam             | 0.0015               | 0.0129    

--- TRACING EPISTEMIC PATH: Chaos ➔ Order ---
STEP  | WORD            | ENTROPY (Resistance) | SCORE (Efficiency)
--------------

In [ ]:
"""
PROJECT NIKA: PHASE 6-B - THE UNSHACKLED NAVIGATOR (FINAL TUNING)
================================================================
"The Minimal Cognitive Path"

CHANGES:
1. Prompt: List-based ("Concept: X, Y...") to kill grammar.
2. Filter: Hard ban on stopwords.
3. Logic: Weighted Entropy (Risk Tolerance).
"""

import torch
import torch.nn.functional as F

# ============================================================================
# SECTION 1: GLOBAL SETUP
# ============================================================================

# Common "Fog Words" that hide the geodesic
STOPWORDS = {
    "the", "of", "and", "to", "a", "in", "is", "that", "it", "as", "was", "for",
    "with", "on", "by", "this", "are", "be", "or", "at", "from", "an", "which",
    "concept", "implies", "leads", "more", "most", "some", "any", "all",
    "connect", "connection", "between", "relating", "related"
}

class TunedNavigator:
    def __init__(self):
        # Auto-load or Reuse
        if 'model' in globals() and 'tokenizer' in globals():
            self.model = globals()['model']
            self.tokenizer = globals()['tokenizer']
        else:
            # Fallback (Should be loaded from previous cells)
            from transformers import AutoTokenizer, AutoModelForCausalLM
            self.tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct")
            self.model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-7B-Instruct", device_map="auto", torch_dtype=torch.float16)

        self.device = self.model.device
        print(f"✓ Tuned Navigator Ready on {self.device}")

    def get_embedding(self, word):
        inputs = self.tokenizer(word, return_tensors="pt").to(self.device)
        with torch.no_grad():
            outputs = self.model(**inputs, output_hidden_states=True)
        return outputs.hidden_states[-1][0, -1, :].float()

    def calculate_entropy(self, probs):
        return -torch.sum(probs * torch.log(probs + 1e-9)).item()

    def find_geodesic(self, start_concept, target_concept, max_steps=8):
        print(f"\n--- GEODESIC: {start_concept} ➔ {target_concept} ---")
        print(f"{'STEP':<5} | {'WORD':<15} | {'ENTROPY':<10} | {'SCORE':<10}")
        print("-" * 55)

        path = [start_concept]
        current_word = start_concept
        target_vec = F.normalize(self.get_embedding(target_concept), dim=0)

        for step in range(max_steps):
            # NEW PROMPT: List context forces direct associations
            prompt = f"Concepts: {start_concept}, {current_word},"

            inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
            with torch.no_grad():
                outputs = self.model(**inputs)

            logits = outputs.logits[0, -1, :].float()
            probs = F.softmax(logits, dim=-1)

            top_k_probs, top_k_ids = torch.topk(probs, 100) # Deeper search

            best_word = None
            best_score = -float('inf')
            sel_entropy = 0.0

            for i, token_id in enumerate(top_k_ids):
                word = self.tokenizer.decode(token_id).strip()
                word_clean = word.lower().replace(",", "").replace(".", "")

                # STRICT FILTERING
                if (not word_clean.isalpha()) or (len(word_clean) < 3): continue
                if word_clean in STOPWORDS: continue
                if word_clean in [w.lower() for w in path]: continue

                # METRICS
                cand_vec = F.normalize(self.get_embedding(word_clean), dim=0)

                # 1. Alignment (Cosine)
                alignment = torch.dot(cand_vec, target_vec).item()

                # 2. Probability (Manifold)
                prob = top_k_probs[i].item()

                # 3. Entropy (Lookahead Risk)
                # We relax the check here to avoid "Night->Darkness" trap
                # We assume High Prob in a List Context = Low Cognitive Resistance
                resistance = 1.0 - prob # Simple resistance metric

                # TUNED FORMULA:
                # We weight Alignment heavily (0.7) to force movement
                # We weight Probability (0.3) to keep it natural
                # We divide by Resistance to penalize random jumps
                score = (alignment * 0.7 + prob * 0.3) / (resistance + 0.5)

                if score > best_score:
                    best_score = score
                    best_word = word_clean
                    sel_entropy = resistance

            if not best_word: break

            path.append(best_word)
            current_word = best_word
            print(f" {step+1:<5} | {best_word:<15} | {sel_entropy:<10.4f} | {best_score:<10.4f}")

            # Check for semantic convergence (Cosine > 0.85)
            curr_vec = F.normalize(self.get_embedding(current_word), dim=0)
            if torch.dot(curr_vec, target_vec).item() > 0.85:
                print(f" >> CONVERGENCE DETECTED ({current_word} ≈ {target_concept})")
                break

            if current_word == target_concept.lower(): break

        return path

# ============================================================================
# EXECUTION
# ============================================================================

if __name__ == "__main__":
    nav = TunedNavigator()

    # 1. Logic -> Emotion (The Hard Problem)
    nav.find_geodesic("Logic", "Emotion")

    # 2. Chaos -> Order (The Abstract Problem)
    nav.find_geodesic("Chaos", "Order")

    # 3. Night -> Day (The Intuitive Problem)
    nav.find_geodesic("Night", "Day")

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✓ Tuned Navigator Ready on cuda:0

--- GEODESIC: Logic ➔ Emotion ---
STEP  | WORD            | ENTROPY    | SCORE     
-------------------------------------------------------
 1     | mathematics     | 0.9310     | 0.3015    
 2     | relation        | 0.9939     | 0.3220    
 3     | structure       | 0.9032     | 0.3233    
 4     | communication   | 0.9982     | 0.3247    
 5     | culture         | 0.9977     | 0.3232    
 6     | belief          | 0.9920     | 0.3220    
 7     | consciousness   | 0.9983     | 0.3072    
 8     | sense           | 0.9984     | 0.3172    

--- GEODESIC: Chaos ➔ Order ---
STEP  | WORD            | ENTROPY    | SCORE     
-------------------------------------------------------
 1     | order           | 0.9452     | 0.3595    

--- GEODESIC: Night ➔ Day ---
STEP  | WORD            | ENTROPY    | SCORE     
-------------------------------------------------------
 1     | sky             | 0.8280     | 0.4239    
 2     | star            | 0.6899     |